In [1]:
import cv2
import numpy as np
import pafy

In [2]:
#選取Youtube影片並創建一個對象“播放”，其中包含webm / mp4格式的影片的最佳分辨率
url = 'https://www.youtube.com/watch?v=V2hlQkVJZhE'
vPafy = pafy.new(url)
play = vPafy.getbest(preftype="mp4")

In [3]:
#控制影片截取(可透過相機截取 ,play.url改 0表示一台)
cap = cv2.VideoCapture(play.url)
#設置影片幀的高度和寬度
#cap.set(3, 480) #set width of the frame
#cap.set(4, 640) #set height of the frame

In [4]:
#創建3個list 存 Model_Mean_Values,Age和Gender
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
gender_list = ['Male', 'Female']

In [5]:
#定義一個fuction來加載年齡和性別檢測器的caffemodel和prototxt，這些基本上是預先訓練的CNN模型將進行檢測
def load_caffe_models():
    age_net = cv2.dnn.readNetFromCaffe('deploy_age.prototxt', 'age_net.caffemodel')
    gender_net = cv2.dnn.readNetFromCaffe('deploy_gender.prototxt', 'gender_net.caffemodel')
    return(age_net, gender_net)

In [6]:
if __name__ == "__main__":
    age_net, gender_net = load_caffe_models()
    

In [7]:
#cap.read（）返回一個bool（True / False）。如果正確讀取幀，則為True
while True:
    ret, image = cap.read()
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    
    if(len(faces)>0):
        print("Found {} faces".format(str(len(faces))))

    for (x, y, w, h )in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 0), 2)
        #Get Face 
        face_img = image[y:y+h, h:h+w].copy()
    
        blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        #Predict Gender
        gender_net.setInput(blob)
        gender_preds = gender_net.forward()
        gender = gender_list[gender_preds[0].argmax()]
        print("Gender : " + gender)
        #Predict Age
        age_net.setInput(blob)
        age_preds = age_net.forward()
        age = age_list[age_preds[0].argmax()]
        print("Age Range: " + age)
    
        overlay_text = "%s %s" % (gender, age)
        cv2.putText(image, overlay_text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('frame', image)
        #0xFF is a hexadecimal constant which is 11111111 in binary.
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
# 釋放攝影機
cap.release()

# 關閉所有 OpenCV 視窗
cv2.destroyAllWindows()


Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (38, 43)


Found 1 faces
Gender : Male
Age Range: (38, 43)
Found 1 faces
Gender : Female
Age Range: (48, 53)
Found 1 faces
Gender : Female
Age Range: (48, 53)
Found 1 faces
Gender : Female
Age Range: (48, 53)
Found 1 faces
Gender : Female
Age Range: (48, 53)
Found 1 faces
Gender : Male
Age Range: (48, 53)
Found 1 faces
Gender : Female
Age Range: (48, 53)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Female
Age Range: (8, 12)
Found 1 faces
Gender : Female
Age Range: (4, 6)
Found 1 faces
Gender : Female
Age Range: (0, 2)
Found 1 faces
Gender : Male
Age Range: (4, 6)
Found 1 faces
Gender : Male
Age Range: (0, 2)
Found 1 faces
Gender : Female
Age Range: (0, 2)
Found 1 faces
Gender : Male
Age Range: (0, 2)
Found 1 faces
Gender : Male
Age Range: (4, 6)
Found 1 faces
Gender : Male
Age Range: (4, 6)
Found 1 faces
Gender : Male
Age Range: (4, 6)
Found 1 faces
Gender : Male
Age Range: (0, 2)
Found 1 faces
Gender : Male
Age Range: (0, 2)
Found 1 faces
Gender : Male
Age Range: (0, 

KeyboardInterrupt: 